In [32]:
import numpy as np
import tensorflow as tf
from string import punctuation
from collections import Counter
import collections
import sklearn as sk

In [2]:

with open('rev_3.txt', 'r') as f:
    reviews = f.read()
with open('lab.txt', 'r') as f:
    labels_org = f.read()

In [3]:
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')
all_text = ' '.join(reviews)
words = all_text.split()

In [5]:
vocabulary_size = 400
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    i=0
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, count, vocab_to_int, int_to_vocab = build_dataset(words,
                                                            vocabulary_size)
reviews_ints=[]
for each in reviews:
    reviews_ints.append([vocab_to_int.get(word,0) for word in each.split()])
reviews_ints[0:10]

[[312, 0, 5, 11, 391, 12, 20, 46, 392, 140, 65, 0, 4, 393, 24],
 [5, 0, 0, 313, 82, 212, 0],
 [5, 0, 394, 0, 92, 0, 0, 4, 314, 6],
 [188, 213, 214],
 [5, 247, 189, 395, 5, 247, 189],
 [0, 58, 3, 5, 0, 11, 3, 0, 0, 0, 0, 315, 0, 0, 13, 0, 11, 0, 0],
 [5, 190, 0, 3, 0, 30, 0, 0, 22, 0, 215],
 [5, 190, 4, 0, 3, 0, 215, 22, 3, 0, 9, 0],
 [0, 0, 113, 55, 5, 0, 0, 93],
 [5, 53, 0, 0, 164, 8, 165, 0, 0, 0]]

In [17]:
#labels = np.array([1 if l == "positive" else 0 for l in labels_org.split()])
labels = np.array([int(_)  for _ in labels_org.split()]).astype(np.int32)+1
print(len(labels))
print(labels[:10])

1000
[2 2 2 2 2 0 2 2 2 2]


In [18]:
A=[len(x) for x in reviews_ints]
review_lens = Counter(A)
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 30


In [19]:
reviews_ints = [r[0:30] for r in reviews_ints if len(r) > 0]
print(len(reviews_ints),reviews_ints[0])
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

1000 [312, 0, 5, 11, 391, 12, 20, 46, 392, 140, 65, 0, 4, 393, 24]
Zero-length reviews: 0
Maximum review length: 30


In [20]:
seq_len = 30
features = np.zeros((len(reviews_ints), seq_len), dtype=int)
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
features[0,:]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 312,   0,   5,  11, 391,  12,  20,  46, 392, 140,  65,
         0,   4, 393,  24])

In [21]:
T=[int_to_vocab.get(features[0,i],'UNK') for i in range(30)]
print(T)
del T

['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'Which', 'UNK', 'BMW', 'is', 'right', 'for', 'you', 'We', 'test', 'them', 'all', 'UNK', 'to', 'find', 'out']


In [22]:
from sklearn.model_selection import train_test_split

train_x,XX,train_y,YY=train_test_split(features,labels,test_size=0.2)
val_x, test_x,val_y, test_y=train_test_split(XX,YY,test_size=0.5)

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nValidation label set: \t{}".format(val_y.shape),
      "\nTest label set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(800, 30) 
Validation set: 	(100, 30) 
Test set: 		(100, 30)
label set: 		(800,) 
Validation label set: 	(100,) 
Test label set: 		(100,)


In [23]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

400


In [47]:
lstm_size = 22
lstm_layers = 1
batch_size = 50
learning_rate = 0.1
embed_size = 30 
n_words = len(vocab_to_int) #+ 1 # Add 1 for 0 added to vocab
print(n_words)
# Create the graph object
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, 30], name="inputs")
    labels_ = tf.placeholder(tf.int32, [None,], name="labels")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

def lstm_cell():
    #lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    lstm=tf.contrib.rnn.BasicRNNCell(num_units=lstm_size,reuse=tf.get_variable_scope().reuse)
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
with tf.name_scope("RNN_layers"):
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32) 
    
with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)  
    
with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 3, activation_fn=None)
    soft_a=tf.nn.softmax(predictions)
with tf.name_scope('cost'):
    #cost = tf.losses.mean_squared_error(labels_, predictions)
    cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=predictions))
    #cost=tf.reduce_mean(tf.squared_difference(labels_,predictions))
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
with tf.name_scope('validation'):
    #correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    #accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    correct = tf.nn.in_top_k(predictions,labels_ , 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    #total_error = tf.reduce_sum(tf.square(labels_ - tf.reduce_mean(labels_)))
    #unexplained_error = tf.reduce_sum(tf.square(labels_ - predictions))
    #accuracy = 1- tf.div(total_error, unexplained_error)
    
tf.summary.histogram('predictions', predictions)
tf.summary.scalar('cost', cost)
merged = tf.summary.merge_all()

epochs = 200
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, "checkpoints/sentiment_2.ckpt")
    train_writer = tf.summary.FileWriter('train_I_II', sess.graph)
    test_writer = tf.summary.FileWriter('test_I_II', sess.graph)
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: np.array(y[:, None]).reshape(-1),
                    keep_prob: 1,
                    initial_state: state}
            summary, loss, state, _ = sess.run([merged, cost, final_state, optimizer], feed_dict=feed)
#            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)

            train_writer.add_summary(summary, iteration)
        
            if iteration%10==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%10==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: np.array(y[:, None]).reshape(-1),
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state,rr = sess.run([accuracy, final_state,soft_a], feed_dict=feed)
                    summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                    y_pred=np.argmax(rr,1)
                print("Val acc: {:.3f} ".format(np.mean(val_acc)))
                a=sk.metrics.f1_score(np.array(y[:,None]).reshape(-1),y_pred,average='micro')
                print(a)
                if (a>0.62):
                    saver.save(sess, "checkpoints/sentiment_3_RNNenbedo_.ckpt")
                    break
            iteration +=1
            test_writer.add_summary(summary, iteration)
            saver.save(sess, "checkpoints/sentiment_tempo_.ckpt")

400
Epoch: 0/200 Iteration: 10 Train loss: 1.183
Val acc: 0.580 
0.6
Epoch: 1/200 Iteration: 20 Train loss: 1.097
Val acc: 0.470 
0.4
Epoch: 1/200 Iteration: 30 Train loss: 0.801
Val acc: 0.550 
0.54
Epoch: 2/200 Iteration: 40 Train loss: 0.921
Val acc: 0.550 
0.56
Epoch: 3/200 Iteration: 50 Train loss: 0.838
Val acc: 0.580 
0.6
Epoch: 3/200 Iteration: 60 Train loss: 0.950
Val acc: 0.580 
0.6
Epoch: 4/200 Iteration: 70 Train loss: 1.003
Val acc: 0.590 
0.62
Epoch: 4/200 Iteration: 80 Train loss: 0.738
Val acc: 0.590 
0.6
Epoch: 5/200 Iteration: 90 Train loss: 0.846
Val acc: 0.580 
0.62
Epoch: 6/200 Iteration: 100 Train loss: 0.695
Val acc: 0.580 
0.62
Epoch: 6/200 Iteration: 110 Train loss: 0.746
Val acc: 0.600 
0.62
Epoch: 7/200 Iteration: 120 Train loss: 0.718
Val acc: 0.600 
0.64
Epoch: 8/200 Iteration: 130 Train loss: 0.701
Val acc: 0.570 
0.58
Epoch: 8/200 Iteration: 140 Train loss: 0.946
Val acc: 0.530 
0.52
Epoch: 9/200 Iteration: 150 Train loss: 0.903
Val acc: 0.440 
0.46
Epoch

Epoch: 76/200 Iteration: 1220 Train loss: 0.809
Val acc: 0.570 
0.58
Epoch: 76/200 Iteration: 1230 Train loss: 0.640
Val acc: 0.570 
0.6
Epoch: 77/200 Iteration: 1240 Train loss: 0.852
Val acc: 0.570 
0.56
Epoch: 78/200 Iteration: 1250 Train loss: 0.721
Val acc: 0.580 
0.58
Epoch: 78/200 Iteration: 1260 Train loss: 0.824
Val acc: 0.590 
0.6
Epoch: 79/200 Iteration: 1270 Train loss: 0.914
Val acc: 0.590 
0.58
Epoch: 79/200 Iteration: 1280 Train loss: 0.694
Val acc: 0.580 
0.6
Epoch: 80/200 Iteration: 1290 Train loss: 0.765
Val acc: 0.590 
0.6
Epoch: 81/200 Iteration: 1300 Train loss: 0.750
Val acc: 0.580 
0.58
Epoch: 81/200 Iteration: 1310 Train loss: 0.677
Val acc: 0.590 
0.6
Epoch: 82/200 Iteration: 1320 Train loss: 0.778
Val acc: 0.590 
0.6
Epoch: 83/200 Iteration: 1330 Train loss: 0.694
Val acc: 0.590 
0.6
Epoch: 83/200 Iteration: 1340 Train loss: 0.871
Val acc: 0.570 
0.56
Epoch: 84/200 Iteration: 1350 Train loss: 0.825
Val acc: 0.580 
0.6
Epoch: 84/200 Iteration: 1360 Train loss: 

Epoch: 150/200 Iteration: 2410 Train loss: 0.781
Val acc: 0.590 
0.62
Epoch: 151/200 Iteration: 2420 Train loss: 0.732
Val acc: 0.590 
0.6
Epoch: 151/200 Iteration: 2430 Train loss: 0.649
Val acc: 0.570 
0.56
Epoch: 152/200 Iteration: 2440 Train loss: 0.752
Val acc: 0.560 
0.58
Epoch: 153/200 Iteration: 2450 Train loss: 0.737
Val acc: 0.570 
0.56
Epoch: 153/200 Iteration: 2460 Train loss: 0.837
Val acc: 0.570 
0.56
Epoch: 154/200 Iteration: 2470 Train loss: 0.904
Val acc: 0.560 
0.56
Epoch: 154/200 Iteration: 2480 Train loss: 0.710
Val acc: 0.570 
0.58
Epoch: 155/200 Iteration: 2490 Train loss: 0.770
Val acc: 0.570 
0.58
Epoch: 156/200 Iteration: 2500 Train loss: 0.725
Val acc: 0.570 
0.56
Epoch: 156/200 Iteration: 2510 Train loss: 0.643
Val acc: 0.570 
0.56
Epoch: 157/200 Iteration: 2520 Train loss: 0.741
Val acc: 0.560 
0.58
Epoch: 158/200 Iteration: 2530 Train loss: 0.735
Val acc: 0.570 
0.56
Epoch: 158/200 Iteration: 2540 Train loss: 0.829
Val acc: 0.570 
0.56
Epoch: 159/200 Iterat

In [ ]:
test_acc = []
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/sentiment_3_RNNenbedo_.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: np.array(y[:, None]).reshape(-1),
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))